In [202]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import gc
from pathlib import Path
from tqdm import tqdm

In [203]:
from datetime import datetime, timedelta
import similaripy as sim

In [294]:
data = pd.read_excel('')
data

,name,restaurant,content,type,address,date,type2,ad1
0,에삐S2,멕시칼리,빠빠에 아래에 깔린 치즈 덕분인지 똑같은 소고기일텐데 소고기타코보다 빠빠에 들어있는...,"멕시코,남미음식",서울특별시 광진구 능동,12.30.토,남미음식,서울특별시
1,에삐S2,롯데리아 구의역점,요즘 롯데리아 갈때마다 먹는 메뉴 '깡돼후'\n무뼈에 달달한편인데 핫소스에 찍어먹으...,햄버거,서울특별시 광진구 자양동,12.29.금,패스트푸드,서울특별시
2,에삐S2,커피랩스로스터리 합정점,2층이 넓은 편이라 좋았어요.\n이야기하기 좋은 카페!,브런치,서울특별시 마포구 서교동,12.29.금,브런치,서울특별시
3,에삐S2,23플래터,크림리조또 위에 토핑된 청양고추 덕분인지 살짝 매콤한맛도 나면서 맛있었어요~\n매장...,이탈리아음식,서울특별시 마포구 서교동,12.29.금,양식,서울특별시
4,에삐S2,썬키친,👏👏👏👏👏 \n맛집 성공해서 기분이 좋았어요.\n고등어덮밥이랑 크림버섯누룽지 두 메...,요리주점,서울특별시 마포구 합정동,12.28.목,주점,서울특별시
...,...,...,...,...,...,...,...,...
42692,노아517,포담 서촌본점,윤기가 가득한 홍초 탕수육 짱맛이에여\n다른 곳에서 보기 힘든 메뉴가 더 특별합니다,중식당,서울특별시 종로구 적선동,11.15.수,중식,서울특별시
42693,노아517,포담 서촌본점,튀김옷이 두껍지않고 맛있는 탕수육 추천,중식당,서울특별시 종로구 적선동,10.9.월,중식,서울특별시
42694,노아517,월양,너무 후다닥 먹어서 고기 사진이 없는데\n고기는 대파 양파 등등이랑 직접 구워주시고...,양갈비,서울특별시 영등포구 당산동5가,8.19.토,고기,서울특별시
42695,노아517,포담 서촌본점,"홍초 탕수육, 탄탄면 맛있어요",중식당,서울특별시 종로구 적선동,8.8.화,중식,서울특별시


### Train Test split

In [295]:
train_data =  data.copy()
test_data = data.groupby('name').head(2)
test_index = data.groupby('name').head(2).index
train_data = train_data.drop(test_index)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

### Item Collaborative Filtering

Interaction Matrix 

In [296]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity


#라벨 인코딩 적용
user_encoder = LabelEncoder()
type_encoder = LabelEncoder()


#type이 14개일때
#train_data['user_id'] = user_encoder.fit_transform(train_data['name'])
#train_data['type_id'] = type_encoder.fit_transform(train_data['type2'])
#type_mapping = dict(zip(train_data['type_id'], train_data['type2']))

#type이 130개 일때
train_data['user_id'] = user_encoder.fit_transform(train_data['name'])
train_data['type_id'] = type_encoder.fit_transform(train_data['type'])
type_mapping = dict(zip(train_data['type_id'], train_data['type']))

#user-item interaction matrix
interaction_matrix = pd.pivot_table(train_data, index='user_id', columns='type_id', aggfunc='size', fill_value=0)
interaction_matrix.columns = [f"type_id:{type_id} ({type_mapping[type_id]})"
                              for type_id in interaction_matrix.columns]


interaction_matrix.head(30)

,type_id:0 (101번지남산돈까스),type_id:1 (가야밀면),type_id:2 (갈비탕),type_id:3 (감자탕),type_id:4 (게요리),type_id:5 (고기뷔페),"type_id:6 (곰탕,설렁탕)","type_id:7 (곱창,막창,양)",type_id:8 (국밥),type_id:9 (국수),...,type_id:103 (푸드코트),type_id:104 (프랑스음식),type_id:105 (피자),type_id:106 (한식),type_id:107 (한식뷔페),type_id:108 (한정식),type_id:109 (핫도그),"type_id:110 (해물,생선요리)",type_id:111 (해장국),type_id:112 (햄버거)
user_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,2,1,...,0,0,0,17,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
interaction_matrix.shape

(3057, 14)

In [297]:
#빈도 행렬
interaction_matrix_ = interaction_matrix.transpose()

#코사인 유사도
res_sim = cosine_similarity(interaction_matrix_, interaction_matrix_)

#코사인 유사도 데이터프레임
type_sim_df = pd.DataFrame(data=res_sim, index=interaction_matrix_.index, columns=interaction_matrix_.index)
type_sim_df.shape

(113, 113)

In [299]:
#유저가 먹은 레스토랑 타입 시퀀스
#user_history = np.array(interaction_matrix.loc[6]).reshape(1,14)
user_history = np.array(interaction_matrix.loc[6]).reshape(1,113)
user_history.shape

(1, 113)

In [302]:
def inference(interaction_matrix,type_sim_df, index):
    
    #유저가 먹은 레스토랑 타입 시퀀스
    user_history = np.array(interaction_matrix.loc[index]).reshape(1,113)

    #유사도 계산
    test_inference = np.dot(user_history,np.array(type_sim_df,dtype=float))

    #top5
    top5_indices = np.argsort(-test_inference,axis=1)

    #최종 결과
    result = interaction_matrix.columns[top5_indices[0][0:5]]

    #정답
    #label = list(test_data.loc[(index-1)*2: index*2-1]['type2'])
    label = list(test_data.loc[(index-1)*2: index*2-1]['type'])

    result_list = [] 
    for i in range(len(result)):
        result_list.append(result[i].split(' ')[1].split('(')[1].split(')')[0])
    result_list
            

    #정확도 계산
    acc = 0
    for i in label:
        if i in result_list:
            acc+=1

    acc = acc/2
    
    return acc

In [303]:
#정확도 계산

acc_sum = 0 
for i in range(len(interaction_matrix)):
    acc_sum += inference(interaction_matrix,type_sim_df,i)
acc_sum/len(interaction_matrix)

0.21131828590121032